# Python Requests File Query

Create, Upload, Query, Download CSV file in other workspaces

## Imports

Import necessary python modules.

In [1]:
import os
import requests
import urllib3
from datetime import datetime
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Define Web Service Command Syntax for Requests

In [2]:
class WebServices(object):

    def __init__(self, host_url="", session=None):
        self.host_url = host_url
        self.session = session if session != None else requests.Session()

    def set_api_key(self, api_key):
        self.headers = {'x-ni-api-key':api_key}
        self.auth = ""
        
    def set_usr_pwd(self, username, password):
        self.headers = ""
        self.auth = (username, password)
    
    def post_route(self, url, post_json="", post_files=""):
        print("POST " + self.host_url + url)
        response = self.session.post(self.host_url+url, json=post_json, files=post_files, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response
    
    def get_route(self, url):
        print("GET " + self.host_url + url)
        response = self.session.get(self.host_url+url, json={}, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

    def put_route(self, url, put_json):
        print("PUT " + self.host_url + url)
        response = self.session.put(self.host_url+url, json=put_json, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

## File I/O Functions

In [3]:
def make_local_file(now):
    now_str = str(now)
    file_name = "Example File.csv"
    file_path = file_name # file paths are by default relative to the running notebook file in JupyterHub
    f = open(file_path, "w", encoding="utf-8")
    f.write(now_str)
    f.close()
    print(str('Created "' + file_name + '" at ' + now_str))
    return file_path, file_name

In [4]:
def upload_file(webServices, file_path, file_name, workspace):
    f = open(file_path, 'r')
    file_content = f.read()
    f.close()
    files = {'file': (file_name, file_content)}
    if workspace != None and workspace != "":
        response = webServices.post_route("nifile/v1/service-groups/Default/upload-files?workspace=" + workspace, post_files=files)
    else: # Default workspace
        response = webServices.post_route("nifile/v1/service-groups/Default/upload-files", post_files=files)
    response_json = response.json()
    print(str(response_json))
    file_id = str(response_json['uri'])[-24:]
    return file_id

In [5]:
def query_files(webServices, file_name, file_id, workspace):
    skip = 0
    take = 1
    query_json = {"idsQuery": {"operation":"EQUAL", "ids":[file_id]},
                  "extensionQuery": {"operation":"EQUAL", "value":"csv"},
                  "sizeMaxQuery": {"operation":"LESS_THAN", "value":5000},
                  "sizeMinQuery": {"operation":"GREATER_THAN", "value":10},
                  "createdQuery": {"operation":"GREATER_THAN", "value": "2018-05-14T00:00:00.000Z"},
                  "propertiesQuery": [{"key":"Name", "operation":"CONTAINS", "value":file_name}]
                 }
    url = "nifile/v1/service-groups/Default/query-files?" + str(skip) + "=0&take=" + str(take)
    if workspace != None and workspace != "": url = url + "&workspace=" + workspace
    response = webServices.post_route(url, post_json=query_json)
    return response.json()

In [6]:
def file_io(webServices): # assumes you are ONLY using the Default workspace
    workspace = None # Default workspace
    #workspace = "93f869bd-f4df-4fb6-84e3-0499cae06a7c" # workspace != Default , workspace = foo
    now = datetime.today().isoformat() # "2019-09-05T18:45:08Z"
    file_path, file_name = make_local_file(now)
    print(file_path, file_name)
    file_id = upload_file(webServices, file_path, file_name, workspace)
    print(file_id)
    file = query_files(webServices, file_name, file_id, workspace)
    print(file)
    return file_id

## Connect to SystemLink, Run File I/O, Close Connection

In [7]:
# Connection Info
host_url = "https://localhost:443/"
host_url = os.getenv("SYSTEMLINK_HTTP_URI") # available only in JupyterHub on SystemLink server
api_key = os.getenv("SYSTEMLINK_API_KEY")   # available only in JupyterHub on SystemLink server
print(str(api_key))

# Return WebServices object (set host_url)
webServices = WebServices(host_url)

# Define the connection credentials using an ApiKey or username/password
webServices.set_api_key(api_key)
#webServices.set_usr_pwd("admin", "mypassword")

# perform tag operations
file_io(webServices)
print("done")

wSo_Orl4sRY5a0n922pDjsFp6kPlKFhl2kdFFQQ5Gz
Created "Example File.csv" at 2023-05-10T15:56:48.527106
Example File.csv Example File.csv
POST https://bradt2-lt/nifile/v1/service-groups/Default/upload-files?workspace=93f869bd-f4df-4fb6-84e3-0499cae06a7c
<Response [201]>
{'uri': '/nifile/v1/service-groups/Default/files/645c0510e49e4dd11462c2c9'}
645c0510e49e4dd11462c2c9
POST https://bradt2-lt/nifile/v1/service-groups/Default/query-files?0=0&take=1&workspace=93f869bd-f4df-4fb6-84e3-0499cae06a7c
<Response [200]>
{'_links': {'self': {'href': '/nifile/v1/service-groups/Default/query-files'}, 'search': {'href': '/nifile/v1/service-groups/Default/files'}, 'query': {'href': '/nifile/v1/service-groups/Default/query-files'}, 'upload': {'href': '/nifile/v1/service-groups/Default/upload-files'}, 'deleteFiles': {'href': '/nifile/v1/service-groups/Default/delete-files'}}, 'availableFiles': [{'_links': {'self': {'href': '/nifile/v1/service-groups/Default/files/645c0510e49e4dd11462c2c9'}, 'data': {'href':